# spectometric data from asteroids used for the smass classification

In [8]:
# import libraries and define functions.

import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
import xgboost as xgb


def correct_0(df):
    """cleans the data in various ways, like resetting the first row,
    splitting columns and converting strings to ints or floats,
    note that the column names arent necessary accurate."""
    zero = df.columns[0]
    zero_df = pd.DataFrame([zero], columns=[zero])
    df = pd.concat([zero_df, df])
    df = df.reset_index(drop=True)
    df['wavelength'] = df[zero].str[:-13]
    df['wavelength'] = df['wavelength'].str.strip(' ')
    df['reflectance'] = df[zero].str[-13:-6]
    df['reflectance'] = df['reflectance'].str.strip(' ')
    df = df.drop([zero], axis=1)
    return df
    

def translate(df, data, name):
    """takes the individual spectra data for each asteroid and fits it into dataframe
    with each possible filter as a parameter""" 
    data = correct_0(data)
    df.loc[len(df)] = 0
    df.loc[len(df)-1,'ast'] = name
    for i in range(len(data['wavelength'])):
        wave = data['wavelength'][i]
        ref = data['reflectance'][i]
        df.loc[len(df)-1, wave] = ref
    return df
    
    
def type_setter(name):
    # THIS FUNCTION IS NOW OBSOLETE!
    """looks up (name) on wikipedia and if found returns the type of asteroid it is."""
    try:
        result = wikipedia.search(name)[0]
        ast = wikipedia.page(result)
    except:
        return 0
    ast = ast.content
    try:
        if ast.find('X-type') != -1:
            return 'X'
        if ast.find('C-type') != -1:
            return 'C'
        if ast.find('S-type') != -1:
            return 'S'
        if ast.find('A-type') != -1:
            return 'A'
        if ast.find('B-type') != -1:
            return 'B'
        if ast.find('D-type') != -1:
            return 'D'
        if ast.find('Q-type') != -1:
            return 'Q'
        if ast.find('R-type') != -1:
            return 'R'
        if ast.find('T-type') != -1:
            return 'T'
        if ast.find('V-type') != -1:
            return 'V'
        if ast.find('K-type') != -1:
            return 'K'
        if ast.find('L-type') != -1:
            return 'L'
        if ast.find('O-type') != -1:
            return 'O'
    except: 
        return 0    

def split_data(df):
    """splits the data into a dataset with S, C and Other as labels and another without S and C"""
    df1 = df.copy()
    df2 = df.copy()
    for i in range(len(df1)):
        if df1['type'][i] != 'S':
            if df1['type'][i] != 'C':
                 df1['type'][i] = 'Other'
    for i in range(len(df2)):
        if df2['type'][i] == 'S':
            df2 = df2.drop([i], axis=0)
        elif df2['type'][i] == 'C':
            df2 = df2.drop([i], axis=0)
    return df1, df2


def ast_pred(features, xgbmodel, xgbmodel2):
    """cleans given dataset and uses both XGB models to predict the class of each row"""
    features = features.drop(['ast', 'name', 'type'], axis=1)
    p = xgbmodel.predict(features.values)
    for i in range(len(p)):
        if p[i] == 'Other':
            p[i] = xgbmodel2.predict(features.loc[i:i].values)[0]
    p = pd.DataFrame({'label': p})
    return p 

In [12]:
# run this cell to test a pre-trained model. 

path = 'PATH/TO/FOLDER/1/'

test = pd.read_csv(path+'test.csv')

pred = ast_pred(test, xgbmodel=joblib.load(path+'xgbmodel.pkl'),
                xgbmodel2=joblib.load(path+'xgbmodel2.pkl'))
pred['name'] = test['name']
pred['number'] = test['ast']

pred.sample(3)

,label,name,number
89,S,semiramis,584
50,S,zuchong-zhi,1888
117,X,happelia,578


In [86]:
# run this cell to clean the spectometry data and train a model from scratch. 

path = 'PATH/TO/FOLDER/2/' 

print('-- script inicialized --')

path_to_folders = path + 'EAR_A_I0028_4_SBN0001_SMASSII_V1_0/data'
folders = os.listdir(path_to_folders)
folders = folders[2:]
tables = []
df = pd.DataFrame(columns=['wavelength', 'reflectance'])

# append data from all files to get every unique wavelength.

for folder in folders:
    files = os.listdir(path_to_folders+'/'+folder)
    for file in files:
        if '.tab' in file:
            tables.append(folder+'/'+file)

for file in tables:
    data = pd.read_csv(path_to_folders+'/'+file)
    data = correct_0(data)
    df = df.append(data)
    
    
df = df.reset_index(drop=True)
    

features = df['wavelength'].unique()
features = np.sort(features)
col = np.append(['ast'],features)
smass_dataframe = pd.DataFrame(columns=col)

print('-- dataframe created --')

# load the data from the individual asteroid files.

for file in tables:
    data = pd.read_csv(path_to_folders+'/'+file)
    smass_dataframe = translate(smass_dataframe, data, file)
    
    
    
smass_dataframe['ast'] = pd.Series(smass_dataframe.ast.values.flatten()).str.split('/')
smass_dataframe['ast'] = pd.Series(smass_dataframe.ast.values.flatten()).str[1]
smass_dataframe['ast'] = pd.Series(smass_dataframe.ast.values.flatten()).str.split('_')
smass_dataframe['ast'] = pd.Series(smass_dataframe.ast.values.flatten()).str[0]



smass_dataframe['type'] = 0
smass_dataframe['name'] = 0

print('-- data loaded --')

# prepare and clean the taxonomy file from which labels will be extracted.


tax = pd.read_csv(path+'taxonomy10.csv', error_bad_lines=False)
tax = correct_0(tax)
tax['smass type'] = tax['wavelength'].str[-32]
tax['tholen type'] = tax['wavelength'].str[-16]
tax['??? type'] = tax['wavelength'].str[-8]
tax['last resort type'] = tax['wavelength'].str[-24]
tax['ast'] = tax['wavelength'].str[:19]
tax['ast'] = tax['ast'].str.replace(' ', '')
tax['ast'] = tax['ast'].str.lower()
tax['number'] = tax['ast'].str.strip('abcdefghijklmnopqrstuvwxyz-')
tax['name'] = tax['ast'].str.strip('0123456789-')
tax = tax.drop(['ast'], axis=1)
tax = tax.drop(['wavelength', 'reflectance'], axis=1)

print('-- taxonomy list set --')

# use the new taxonomy dataframe to set labels.
# the nested for loop is important so that each row in smass_dataframe is compared to every row in tax.

for i in range(len(smass_dataframe)):
    for j in range(len(tax)):
        if str(smass_dataframe['ast'].loc[i]) == tax['number'][j]:
            smass_dataframe['name'].iloc[i] = tax['name'][j]
            if tax['smass type'][j] != '-':
                smass_dataframe['type'].iloc[i] = tax['smass type'][j]
            else:
                if tax['tholen type'][j] != '-':
                    smass_dataframe['type'].iloc[i] = tax['tholen type'][j]
                else:
                    if tax['??? type'][j] != '-':
                        smass_dataframe['type'].iloc[i] = tax['??? type'][j]
                    else:
                        smass_dataframe['type'].iloc[i] = tax['last resort type'][j]
                    
smass_dataframe = smass_dataframe.replace(['-', ' '], 0)
for i in range(len(smass_dataframe)):
    if smass_dataframe['type'][i] == 0:
        smass_dataframe = smass_dataframe.drop([i], axis=0)

print('-- labels set --')


# save the dataframe

smass_dataframe.to_csv(path+'smass.csv', index=False)

print('-- dataframe saved --')


# split the data

train, test = train_test_split(smass_dataframe, test_size=0.1)
train = train.reset_index(drop=True)                              
test = test.reset_index(drop=True)
test.to_csv(path+'test.csv', index=False)

print('-- test set ready --')
                          
train1, train2 = split_data(train)


# prepare data for the first xgb classifier
                                  
train_labels = train1.pop("type")
                                  
                                  
train_ast = train1.pop("ast")

train_name = train1.pop("name")

train_features = train1.values

print('-- train set 1 ready --')


# prepare data for the second xgb classifier

train_labels2 = train2.pop("type")


train_ast2 = train2.pop("ast")

train_name2 = train2.pop("name")

train_features2 = train2.values

print('-- train set 2 ready --')


# train models

xgbmodel = xgb.XGBClassifier()
xgbmodel.fit(train_features, train_labels)
joblib.dump(xgbmodel, path+'xgbmodel.pkl', compress=True)
print('-- XGB 1 pickled --')

xgbmodel2 = xgb.XGBClassifier()
xgbmodel2.fit(train_features2, train_labels2)
joblib.dump(xgbmodel2, path+'xgbmodel2.pkl', compress=True)
print('-- XGB 2 pickled --')


# test the model

preds = ast_pred(test, xgbmodel=joblib.load(path+'xgbmodel.pkl'),
                xgbmodel2=joblib.load(path+'xgbmodel2.pkl'))

X = 0
O = 0

for i in range(len(test)):
    if preds['label'][i] == test['type'][i]:
        O += 1
    else:
        X += 1
        
acc = O*100/len(test) 
print('-- accuracy: ', acc,'--')


print('-- script finalized --')

-- script inicialized --
-- dataframe created --
-- data loaded --
-- taxonomy list set --


b'Skipping line 18: expected 1 fields, saw 2\nSkipping line 43: expected 1 fields, saw 2\nSkipping line 73: expected 1 fields, saw 2\nSkipping line 135: expected 1 fields, saw 2\nSkipping line 258: expected 1 fields, saw 2\nSkipping line 1429: expected 1 fields, saw 2\n'


-- labels set --
-- dataframe saved --
-- test set ready --
-- train set 1 ready --
-- train set 2 ready --
-- XGB 1 pickled --
-- XGB 2 pickled --
-- accuracy:  87.21804511278195 --
-- script finalized --
